In [3]:
import os
from libv3.utils import *
import pandas as pd
import json

In [9]:
#### idle trace to which all traces will be adapted in terms of varlist
normal_vpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/normal/varlist_trial6.json'

################ faulty traces
faulty1_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/faulty_data/trace_trial1'
faulty2_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/faulty_data/trace_trial2'
faulty3_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/faulty_data/trace_trial3'
faulty4_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/faulty_data/trace_trial4'
faulty5_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/faulty_data/trace_trial5'
faulty6_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/faulty_data/trace_trial6'

faulty1_vpath = '../trace_data/lora_ducy/single_thread/version_3/faulty_data/varlist_trial1.json'
faulty2_vpath = '../trace_data/lora_ducy/single_thread/version_3/faulty_data/varlist_trial2.json'
faulty3_vpath = '../trace_data/lora_ducy/single_thread/version_3/faulty_data/varlist_trial3.json'
faulty4_vpath = '../trace_data/lora_ducy/single_thread/version_3/faulty_data/varlist_trial4.json'
faulty5_vpath = '../trace_data/lora_ducy/single_thread/version_3/faulty_data/varlist_trial5.json'
faulty6_vpath = '../trace_data/lora_ducy/single_thread/version_3/faulty_data/varlist_trial6.json'


################ normal traces
normal1_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/normal/trace_trial1'
normal2_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/normal/trace_trial2'
normal3_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/normal/trace_trial3'
normal4_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/normal/trace_trial4'
normal5_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/normal/trace_trial5'

normal1_vpath = '../trace_data/lora_ducy/single_thread/version_3/normal/varlist_trial1.json'
normal2_vpath = '../trace_data/lora_ducy/single_thread/version_3/normal/varlist_trial2.json'
normal3_vpath = '../trace_data/lora_ducy/single_thread/version_3/normal/varlist_trial3.json'
normal4_vpath = '../trace_data/lora_ducy/single_thread/version_3/normal/varlist_trial4.json'
normal5_vpath = '../trace_data/lora_ducy/single_thread/version_3/normal/varlist_trial5.json'

In [10]:
#### load varlist for normal trace
#### generate normal_to_number and normal_from_number, to map the events to numbers and vice versa
normal_to_number = read_json(normal_vpath)

if normal_to_number != False:
    normal_from_number = mapint2var(normal_to_number)

In [11]:
#### load faulty1 trace and varlist
paths_to_trace = [
                    faulty1_tpath, 
                    faulty2_tpath, 
                    faulty3_tpath, 
                    faulty4_tpath,
                    faulty5_tpath,
                    faulty6_tpath,
                    normal1_tpath,
                    normal2_tpath,
                    normal3_tpath,
                    normal4_tpath,
                    normal5_tpath,
                ]

paths_to_varlist = [
                    faulty1_vpath,
                    faulty2_vpath, 
                    faulty3_vpath, 
                    faulty4_vpath,
                    faulty5_vpath,
                    faulty6_vpath,
                    normal1_vpath,
                    normal2_vpath,
                    normal3_vpath,
                    normal4_vpath,
                    normal5_vpath,
                    ]




for path_to_varlist, path_to_trace in zip(paths_to_varlist, paths_to_trace):
    ### check if pair of trace and varlist correct
    assert( os.path.basename(path_to_trace).split('_')[1] in path_to_varlist )
    #### check if varlist consistent with normal trace
    check_consistence, _ = is_consistent([normal_vpath, path_to_varlist])

    if check_consistence != False:
        print('varlist is same as normal')
    else:
        print('correcting the trace to work with normal varlist:', path_to_trace)
        faulty_trace = read_traces(path_to_trace)

        faulty_to_number = read_json(path_to_varlist)
        faulty_from_number = mapint2var(faulty_to_number)

        ### get trace with varibale names
        faulty_vartrace = []
        for stateint, ts in faulty_trace:
            # print(stateint, ts)
            state_var = faulty_from_number[stateint]
            faulty_vartrace += [[state_var, ts]]

        ### check if the length of the trace is same. We compare trace with the variable names and trace with the numbers
        assert (len(faulty_vartrace) == len(faulty_trace))

        ### use the varlist same as the normal trace for consistency. If the variable is not in the normal varlist, add it to the normal varlist
        faulty_trace_new = []
        state_var = ''
        state_int = 0
        max_int = max(normal_to_number.values()) ### to increment the normal varlist
        for state_var, ts in faulty_vartrace:
            if state_var in normal_to_number:
                state_int = normal_to_number[state_var]
                faulty_trace_new += [[state_int, ts]]
            else:
                normal_to_number[state_var] = max_int + 1
                max_int += 1
                faulty_trace_new += [[max_int, ts]]

        #### check if the adapted trace is same as the original trace
        assert (len(faulty_trace_new) == len(faulty_trace))

        print('updated varlist with new events:',len(normal_to_number))
        # print(os.path.basename(path_to_trace).split('_')[1])
        # print(os.path.basename(path_to_trace).split('_')[1] in path_to_varlist)
        print('writing varlist to:', path_to_varlist)
        print('writing trace to:', path_to_trace)
        ### save the corrected trace and varlist
        with open(path_to_trace, 'w') as f:
            json.dump(faulty_trace_new, f)

        with open(path_to_varlist, 'w') as f:
            json.dump(normal_to_number, f)

        print('\n')


varlist 1 is not consistent varlist 0
correcting the trace to work with normal varlist: /Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/faulty_data/trace_trial1
updated varlist with new events: 86
writing varlist to: ../trace_data/lora_ducy/single_thread/version_3/faulty_data/varlist_trial1.json
writing trace to: /Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/faulty_data/trace_trial1


varlist 1 is not consistent varlist 0
correcting the trace to work with normal varlist: /Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/faulty_data/trace_trial2
updated varlist with new events: 86
writing varlist to: ../trace_data/lora_ducy/single_thread/version_3/faulty_data/varlist_trial2.json
writing trace to: /Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3

updated varlist with new events: 86
writing varlist to: ../trace_data/lora_ducy/single_thread/version_3/faulty_data/varlist_trial3.json
writing trace to: /Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/faulty_data/trace_trial3


varlist 1 is not consistent varlist 0
correcting the trace to work with normal varlist: /Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/faulty_data/trace_trial4
updated varlist with new events: 86
writing varlist to: ../trace_data/lora_ducy/single_thread/version_3/faulty_data/varlist_trial4.json
writing trace to: /Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version_3/faulty_data/trace_trial4


varlist 1 is not consistent varlist 0
correcting the trace to work with normal varlist: /Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/lora_ducy/single_thread/version

In [12]:
len(normal_from_number.keys())

86

In [13]:
max(normal_to_number.values())

85

In [14]:
normal_from_number

{53: '0-0-0-number',
 36: '0-0-0-noise_detected_counter',
 72: '0-0-0-zero_padding',
 60: '0-0-packet_check-number',
 10: '0-0-0-chrono2',
 9: '0-0-0-chrono1',
 80: '0-0-0-rcv_packet',
 22: '0-0-0-transmissions',
 20: '0-0-0-packet_size',
 63: '0-0-0-cca_time',
 58: '0-0-cca-l',
 48: '0-0-0-transmission_in_pll',
 81: '0-0-0-receiving_data',
 57: '0-0-cca-c',
 84: '0-0-0-ss',
 3: '0-0-0-bluetooth',
 40: '0-0-0-received_full_data',
 59: '0-0-0-channel_status',
 85: '0-0-0-events',
 65: '0-0-0-destination_address',
 0: '0-0-0-lora',
 32: '0-0-0-Tx_time',
 24: '0-0-0-phase_lock_time_saving',
 5: '0-0-0-my_number',
 30: '0-0-0-fast_sleep_time_save',
 26: '0-0-0-failed_attempts',
 74: '0-0-0-rcv_packet1',
 73: '0-0-0-packet',
 52: '0-0-Random-r',
 16: '0-0-0-pll_threshold',
 78: '0-0-0-time_now',
 83: '0-0-0-time_left',
 35: '0-0-0-only_listen',
 37: '0-0-0-ack_data_packets',
 43: '0-0-0-max_wait_time',
 50: '0-0-0-neighbor_adresses',
 64: '0-0-0-send_time_updated',
 18: '0-0-0-cca_interval'